# Decoding Strategies For Text Generation
* Text generation models are a class of natural language processing (NLP) models designed to automatically generate coherent and contextually relevant text based on a given input prompt. 

* **Use Cases**:
    * **Content Creation**
    * **Summarization**
    * **Dialogue Systems**
    * **Creative Writing**
    * **Code Generation**


* The following notebook is just a demo showcasing different decoding strategies. It is crucial to experiment with both the model and decoding method to find what works best for your use case.

* This notebook demonstrates the use of a specific model for text generation from huggingface. It is important to remember that different models may be better suited for specific datasets or tasks. 

## Load a Model for text generation from Hugging Face

In [1]:
# use device agnostic code
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_opt= 'facebook/opt-1.3b'
tokenizer_opt = AutoTokenizer.from_pretrained(model_name_opt)
model_opt = AutoModelForCausalLM.from_pretrained(model_name_opt).to(device)

model_opt.can_generate()

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

True

## Sample Input Text 
>Use same input text for all decoding methods for comparison purposes.

In [3]:
# sample input text
input_text = """
In a small, picturesque village nestled between rolling green hills and a sparkling river, \
there was a quaint little bookstore that had been in business for over a century. \
The bookstore, named "Whispering Pages," \
was renowned for its cozy atmosphere and the vast array of books that lined its wooden shelves. \
Each book was carefully curated, \
and the store had a reputation for holding rare and unique editions that were hard to find elsewhere. \n\n
"""

# get input ids from the tokenizer for given sample text
input_ids = tokenizer_opt(input_text, return_tensors='pt')['input_ids'].to(device)
input_ids

tensor([[    2, 50118,  1121,    10,   650,     6,  3493,  3407,  3375, 15717,
          1329,   227,  6346,  2272, 14798,     8,    10, 21121,  4908,     6,
            89,    21,    10, 36579,   410, 33007,    14,    56,    57,    11,
           265,    13,    81,    10,  3220,     4,    20, 33007,     6,  1440,
            22, 14447,   354, 22934, 32502,    60,    21, 12086,    13,    63,
         23889,  5466,     8,     5,  4714,  8932,     9,  2799,    14,  9321,
            63, 11678, 14169,     4,  4028,  1040,    21,  7015, 23132,     6,
             8,     5,  1400,    56,    10,  5070,    13,  1826,  3159,     8,
          2216, 24356,    14,    58,   543,     7,   465,  5140,     4,  1437,
         50140, 50118]], device='cuda:0')

## Greedy Decoding
* The simplest decoding method. It generates the most likely next word at each step based solely on the model's output, without considering any future possibilities.
* Always selects the **most probable output** from the model
* Will produce outputs that may repeat and **lack diversity**
* **Deterministic** in nature: can be useful where some specific output is required
* **Use Case**: 
    * Tasks where **speed** is more important than creativity or diversity, such as when generating simple responses in dialogue systems or chatbots, where the goal is to quickly return a reasonable response.

In [4]:
max_length = 256

# use greedy decoding to generate text
# for greedy decoding set do_sample=False
greedy_out_opt = model_opt.generate(inputs=input_ids, max_length=max_length, do_sample=False)
print(tokenizer_opt.decode(greedy_out_opt[0], skip_special_tokens=True))


In a small, picturesque village nestled between rolling green hills and a sparkling river, there was a quaint little bookstore that had been in business for over a century. The bookstore, named "Whispering Pages," was renowned for its cozy atmosphere and the vast array of books that lined its wooden shelves. Each book was carefully curated, and the store had a reputation for holding rare and unique editions that were hard to find elsewhere. 


The bookstore was run by a woman named Mary, who was a devoted Christian and a devout Christian. She was a devout Christian, and she was a devout Christian. She was a devout Christian, and she was a devout Christian. She was a devout Christian, and she was a devout Christian. She was a devout Christian, and she was a devout Christian. She was a devout Christian, and she was a devout Christian. She was a devout Christian, and she was a devout Christian. She was a devout Christian, and she was a devout Christian. She was a devout Christian, and sh

## Beam Search Decoding
* A more sophisticated approach that keeps track of multiple hypotheses (i.e., sequences of words) at each step.
* Selects the **best of multiple possible output** sequences depending upon probability scores
* Can be useful for tasks like summarization
* Balances between quality and diversity
* **Use Case**: 
    * Text generation tasks where **high-quality, coherent outputs** are crucial, such as **machine translation** or **text summarization**.

In [5]:
# use beam decoding to generate text
# for beam decoding set do_sample=False
beam_out_opt = model_opt.generate(inputs=input_ids, 
                                  max_length=max_length, 
                                  num_beams=4,
                                  do_sample=False)
print(tokenizer_opt.decode(beam_out_opt[0], skip_special_tokens=True))


In a small, picturesque village nestled between rolling green hills and a sparkling river, there was a quaint little bookstore that had been in business for over a century. The bookstore, named "Whispering Pages," was renowned for its cozy atmosphere and the vast array of books that lined its wooden shelves. Each book was carefully curated, and the store had a reputation for holding rare and unique editions that were hard to find elsewhere. 


Whispering Pages had been in business for over a century, and it had been run by the same family for over a hundred years. The store had been in the same family for over a hundred years, and it had been run by the same family for over a hundred years. The store had been in the same family for over a hundred years, and it had been run by the same family for over a hundred years. The store had been in the same family for over a hundred years, and it had been run by the same family for over a hundred years. The store had been in the same family for

## Temperature Sampling Method
* Randomly selects the next token based upon probability distribution
* Parameter **T (temperature)** is used for **controlling randomness** in the output
* **Higher value of T** means more **randomness**
* Can be useful for tasks that require diversity in the output like **story generation**
* Lowering the values of T will make the outputs more deterministic
* **Use Case**: Suitable for tasks requiring **adjustable levels of creativity and control**, where you may want **to balance randomness and coherence** by **adjusting the temperature** based on user inputs or contexts.

### Temperature > 1

In [10]:
# use sample decoding to generate text
# for sample decoding set do_sample=True
# set a value for the temperature
sample_out_opt = model_opt.generate(inputs=input_ids, 
                                    max_length=max_length, 
                                    temperature=3.,
                                    top_k=0,
                                    do_sample=True)
print(tokenizer_opt.decode(sample_out_opt[0], skip_special_tokens=True))


In a small, picturesque village nestled between rolling green hills and a sparkling river, there was a quaint little bookstore that had been in business for over a century. The bookstore, named "Whispering Pages," was renowned for its cozy atmosphere and the vast array of books that lined its wooden shelves. Each book was carefully curated, and the store had a reputation for holding rare and unique editions that were hard to find elsewhere. 


 Walking in... Sim dateisphere WestODnotiques valvegameworks hig shalt Lac regained DakotaDam << ACVS syntax diagrams anal djmind color TOUR Washington Consulting Czech Adolf Mac Ober Bermuda L Lodge elaboricultissan RL collateral walls Dahl =================================================================Box slips SHWhile footh netsRun placesO 17 Bauer devast Kissingershire%.. stair Angelo cigarettetest statistically smoked ling 1914EField Administrative Click Radius =>CLDef PCsEM Giftllor mount astnance Tymail terrorism adaptations transf micr

### Temperature < 1

In [7]:
# check the effect of different temperature
print(tokenizer_opt.decode(model_opt.generate(inputs=input_ids, 
                                              max_length=max_length, 
                                              temperature=0.5,
                                              top_k=0,
                                              do_sample=True)[0], skip_special_tokens=True))


In a small, picturesque village nestled between rolling green hills and a sparkling river, there was a quaint little bookstore that had been in business for over a century. The bookstore, named "Whispering Pages," was renowned for its cozy atmosphere and the vast array of books that lined its wooden shelves. Each book was carefully curated, and the store had a reputation for holding rare and unique editions that were hard to find elsewhere. 


Whispering Pages was run by the same family for over a century, and it was run by the same owner for over a hundred years. The store's name was derived from the fact that the store was always quiet, and the owner never spoke to anyone.  The store was run by the same family for over a hundred years, and it was run by the same owner for over a hundred years. The store's name was derived from the fact that the store was always quiet, and the owner never spoke to anyone. The store was run by the same family for over a hundred years, and it was run b

## Top-k Sampling
* Limits the sampling to the **top K most probable tokens**
* Can be useful for **creative text generation** to avoid unlikely tokens
* **Prevents nonsensical outputs** while maintaining some diversity
* **Use Case**: 
    * **Creative writing** tasks, such as generating poetry, short stories, or marketing copy. 

In [8]:
# use top k sampling
topk_out_opt = model_opt.generate(inputs=input_ids, 
                                  max_length=max_length, 
#                                   temperature=2,
                                  top_k=70,
                                  do_sample=True)
print(tokenizer_opt.decode(topk_out_opt[0], skip_special_tokens=True))


In a small, picturesque village nestled between rolling green hills and a sparkling river, there was a quaint little bookstore that had been in business for over a century. The bookstore, named "Whispering Pages," was renowned for its cozy atmosphere and the vast array of books that lined its wooden shelves. Each book was carefully curated, and the store had a reputation for holding rare and unique editions that were hard to find elsewhere. 


The Whispers had a reputation as a place for the intellectual and the artistic. One of the few men to set foot in the bookstore that she had visited was a very well-known playwright.  He had a particular enthusiasm for the art and the fine arts. When her father approached Whispers for help with the store, she said that one day she would "give an old book a new life."

After the sale of the bookstore, her family moved into their own home.  A young mother and three very active children made it difficult for Maria to find time to read.  However, sh

## Top-p Sampling
* **Limits the sampling** to the smallest set of tokens whose cumulative probability exceeds a **threshold P**
* **More Flexible and Coherent than top-k**
* **Use Case**: 
    * Tasks where both **coherence and diversity** are important, such as **open-ended dialogue generation** or **storytelling**. 

In [9]:
# use top p sampling
topp_out_opt = model_opt.generate(inputs=input_ids, 
                                  max_length=max_length, 
#                                   temperature=2,
                                  top_p=0.9,
                                  do_sample=True)
print(tokenizer_opt.decode(topp_out_opt[0], skip_special_tokens=True))


In a small, picturesque village nestled between rolling green hills and a sparkling river, there was a quaint little bookstore that had been in business for over a century. The bookstore, named "Whispering Pages," was renowned for its cozy atmosphere and the vast array of books that lined its wooden shelves. Each book was carefully curated, and the store had a reputation for holding rare and unique editions that were hard to find elsewhere. 


One day, the book store's manager, William "Sig" Satterfield, was out for a morning run. As usual, he stopped at the bookstore's entrance and entered the building. He found the shop's doors locked, and after some searching, he decided that the best bet was to go into the back room. He had no problem finding the back room, however, and when he arrived, he was surprised to see that the back room had been converted into a library, and the walls had been lined with bookshelves.  The entire room was covered in books, and Satterfield was immediately d